# 9.3. Statistical Coding - 10 points

## (3pt) Answer these question:

### 1) Which of these codes cannot be Huffman codes for any probability assignment and why?

- a) {0, 10, 11} - Ano vytvoření stromu je v pořádku.
- b) {00, 01, 10, 110} - Ne strom není optimální.
- c) {01, 10} - Ne strom není optimální.

### 2) Classes of codes. Consider the code {0, 01}.
- a) **Is it prefix code?** - Není 0 je prefixem 01.
- b) **Is it uniquely decodable?** - Je, každé přečtení 0 ukončí čtení a probíhá interpretace. Mám načtenou 0 čtu další 0 intrpretuju co mám načteno..
- c) **It it nonsingular?** Ano je nesingulární. Jsem schopni mít více interpretací.

### 3) Optimal word lengths.
- a) Can l=(1, 2, 2) be the word lengths of a binary Huffman code? - Ano, může. Optimální.
- a) Can l=(2, 2, 3, 3) be the word lengths of a binary Huffman code? - Ne nelze, není optimální.

## (7pt) Programming Assignment
- Implement Huffman encoding - build a Huffman tree and encode the file.
- Use the same tree for decoding.
- Use the same files as in Exercise 1
- Compare the resulting compression to the entropy.
- Do not generate bits, string of 0s and 1s are sufficient.

In [1]:
import os
import sys

def adding_module_path():
    module_path = os.path.abspath(os.path.sep.join([".."]*2))

    if module_path not in sys.path:
        sys.path.append(module_path)

adding_module_path()

In [2]:
from src.load_data import get_dataset
from src.load_data import DataSets
from src.get_probs import get_sorted_probs_as_df
import numpy as np
import pandas as pd
import time
from src.types.table_types import Fields

In [3]:
data_dna, path_dna = get_dataset(DataSets.dna)

Loading C://Users//Vojta//Desktop//iv//AKS//datasets\dna\dna.50MB


In [4]:
sorted_df = get_sorted_probs_as_df(data_dna)

Calculating freqs
End of calculating 6.598030090332031
Calculating probs
End of calculating 0.0


In [5]:
sorted_df

,0
A,2.899787e-01
T,2.897280e-01
C,2.102979e-01
G,2.099745e-01
N,8.907318e-06
\n,6.408691e-06
R,1.296997e-06
Y,1.087189e-06
K,8.773804e-07
W,8.583069e-07


In [6]:
class Node:
    def __init__(self, parent=None, left=None, right=None, prob=None, label=None):
        self.left = left
        self.right = right
        self.parent = parent
        self.prob = prob
        self.label = label

    def __str__(self):
        return f"{self.label} -- {self.prob}"

In [7]:
def process_node(node, alphabet, codeword = ""):
    if len(node.label) == 1:
        alphabet[node.label] = codeword

    if node.right is not None:
        process_node(node.right, alphabet, codeword + "1")

    if node.left is not None:
        process_node(node.left, alphabet, codeword + "0")

In [8]:
def build_encoding_table(tree):
    table = {}
    process_node(tree.root, table, "")
    return table

In [9]:
class HuffmanTree:
    def __init__(self):
        pass

    def get_char_encodings(self, alphabet):
        res = {}

        self.root


    def print_node(self, node, level):
        tabs = "\t" * level
        print(f'{tabs} {node}')

        if node.left is not None:
            self.print_node(node.left, level+1)

        if node.right is not None:
            self.print_node(node.right, level+1)


    def print_tree(self):
        self.print_node(self.root, 0)


    def get_left_right(self, last_two):
        left_label = last_two[0][0]
        left_prob = last_two[0][1]

        right_label = last_two[1][0]
        right_prob = last_two[1][1]

        left, right = list(sorted([(left_label, left_prob), (right_label, right_prob)], key=lambda item: len(item[0])))

        left_label = left[0]
        left_prob = left[1]

        right_label = right[0]
        right_prob = right[1]

        return left_label, left_prob, right_label, right_prob

    def get_nodes(self, existing_nodes, last_two):
        left, right, parent = None, None, None

        left_label = last_two[0][0]
        left_prob = last_two[0][1]

        right_label = last_two[1][0]
        right_prob = last_two[1][0]


        left_label, left_prob, right_label, right_prob = self.get_left_right(last_two)

        if left_label in existing_nodes:
            left = existing_nodes[left_label]
        else:
            left = Node(None, None, None, left_prob, left_label)
            existing_nodes[left_label] = left


        if right_label in existing_nodes:
            right = existing_nodes[right_label]
        else:
            right = Node(None, None, None, right_prob, right_label)
            existing_nodes[right_label] = right

        new_node_label = "".join(list(sorted([i[0] for i in last_two])))
        new_node_prob = np.sum([i[1] for i in last_two])

        parent = Node(None, left, right, new_node_prob, new_node_label)
        existing_nodes[new_node_label] = parent
        left.parent = parent
        right.parent = parent

        return left, right, parent

    def build_tree(self, data):
        #sort by probability
        sorted_df = get_sorted_probs_as_df(data)

        dic = sorted_df.to_dict()[0]
        existing_nodes = {}
        parent = None

        while len(list(dic.keys())) >= 2:
            sorted_dic = sorted(dic.items(), key=lambda item: item[1], reverse=False)
            last_two = sorted_dic[0:2]
            
            left, right, parent = self.get_nodes(existing_nodes, last_two)


            new_dic = sorted_dic.copy()

            del new_dic[0:2]


            new_dic = dict(new_dic) 
            new_dic[parent.label] = parent.prob

            dic = new_dic
        
        self.root = parent

    def decode_binary(self, binary_text):

        pass
        #binary text

        #go over tree

        #if leaf return char


    def encode_char(self, char):
        #get builded tree

        #go over tree

        #if leaf return binary
        pass

    
    def encode(self, data):
        #iterate over char and create long string
        table = build_encoding_table(self)
        encoded = ""
        for c in data:
            encoded += table[c]
        return encoded

    def decode(self, data):
        decoded = ""
        #iterate over stream and get all decoded values
        ptr = self.root

        for bit in data:
            if 1 == len(ptr.label):
                decoded += ptr.label
                ptr = self.root

            if bit == '1':
                ptr = ptr.right

            if bit == '0':
                ptr = ptr.left
        
        decoded += ptr.label
        
        return decoded

In [10]:
tree = HuffmanTree()

In [11]:
tree.build_tree(data_dna)

Calculating freqs
End of calculating 6.4379966259002686
Calculating probs
End of calculating 0.0


In [12]:
print(tree.root)


BDHVMSKWRYNGCAT -- 1.0


# Small test

In [13]:
test = data_dna[0:10]
test

'GATCAATGAG'

In [14]:
encoded_data = tree.encode(test)
encoded_data

'11001001001010011001110'

In [15]:
encoded_data

'11001001001010011001110'

In [16]:
decoded_data = tree.decode(encoded_data)
decoded_data

'GATCAATGAG'

In [17]:
table = build_encoding_table(tree)

In [18]:
print(tree.root.right)


BDHVMSKWRYNGC -- 0.4202932357788086


In [36]:
print(tree.root.left)

AT -- 0.5797067642211914


It is better to precalculate code table instead of iterating over nodes

In [37]:
pd.DataFrame.from_dict(table, orient='index')

,0
B,111111111111
D,111111111110
H,11111111110
V,1111111110
M,111111110
S,11111110
K,11111101
W,11111100
R,1111101
Y,1111100


# Kuba test

In [20]:
text = 'A'*10+'E'*15+'I'*12+'S'*3+'T'*4+'P'*13+'N'*2
tree = HuffmanTree()
tree.build_tree(text)
table = build_encoding_table(tree)

Calculating freqs
End of calculating 0.0010025501251220703
Calculating probs
End of calculating 0.0


In [21]:
pd.DataFrame.from_dict(table, orient='index')

,0
S,11111
N,11110
T,1110
A,110
E,10
P,01
I,00


# Results

In [38]:
def run(ds, path, name):
    print(path)

    tree = HuffmanTree()
    tic = time.time()
    tree.build_tree(ds)
    toc = time.time()
    building_tree = toc - tic

    n = len(ds)

    tic = time.time()
    encoded_data = tree.encode(ds)
    toc = time.time()

    encoding_data = toc - tic

    tic = time.time()
    decoded_data = tree.decode(encoded_data)
    toc = time.time()

    decoding_data = toc - tic

    return {
        "building_tree": building_tree,
        "encoding_data": encoding_data,
        "decoding_data": decoding_data,
        Fields.Path.value: path,
        Fields.Parameter.value: "",
        Fields.Type.value: "Huffman",
        Fields.Original.value: n * 8,
        Fields.Aprox.value: len(encoded_data),
        Fields.Calculation.value: building_tree + encoding_data + decoding_data,
        Fields.Name.value: name
    }

In [39]:
data_dna, path_dna = get_dataset(DataSets.dna)
data_english, path_english = get_dataset(DataSets.english)

Loading C://Users//proko//Desktop//University//iv//aks//datasets\dna\dna.50MB
Loading C://Users//proko//Desktop//University//iv//aks//datasets\english\english.50MB


In [40]:
datasets = [(data_dna, path_dna, DataSets.dna.value), (data_english, path_english, DataSets.english.value)]

In [41]:
res = {}
counter = 0

for data, path, name in datasets:
    value = run(data, path, name)
    res[counter] = value
    counter += 1

C://Users//proko//Desktop//University//iv//aks//datasets\dna\dna.50MB
Calculating freqs
End of calculating 9.261345624923706
Calculating probs
End of calculating 0.0
C://Users//proko//Desktop//University//iv//aks//datasets\english\english.50MB
Calculating freqs
End of calculating 8.386399984359741
Calculating probs
End of calculating 0.0009522438049316406


In [42]:
result_dataframe = pd.DataFrame.from_dict(res, orient="index")

In [43]:
result_dataframe

,building_tree,encoding_data,decoding_data,Path,Parameter,Type,Original,Aprox,Calculation,Name
0,9.263349,513.536595,136.444840,C://Users//proko//Desktop//University//iv//aks...,,Huffman,419430400,115869903,659.244783,dna
1,8.394352,2162.968342,471.270896,C://Users//proko//Desktop//University//iv//aks...,,Huffman,419430400,239334456,2642.633590,english


In [44]:
from src.save import save_both

In [45]:
save_both(result_dataframe)